In [1]:
# append a new directory to sys.path
import sys
#sys.path.append('../GBM feature extraction/Multiclass problem')
sys.path.append('../../../')

import pandas as pd
import numpy as np
from expermients_functions import stratified_group_k_fold

In [2]:
data_train = pd.read_csv(r'../../../../Data/LPMC_train.csv')
data_test = pd.read_csv(r'../../../../Data/LPMC_test.csv')


#distance in km
data_train['distance'] = data_train['distance']/1000
data_test['distance'] = data_test['distance']/1000

#rename label
label_name = {'travel_mode': 'choice'}
dataset_train = data_train.rename(columns = label_name)
dataset_test = data_test.rename(columns = label_name)

In [69]:

extra_data = dataset_train[['age', 'female', 'start_time_linear', 'day_of_week', 'car_ownership', 'driving_license', 'purpose_B', 'purpose_HBE', 'purpose_HBO', 'purpose_HBW', 'purpose_NHBO', 'fueltype_Average', 'fueltype_Diesel', 'fueltype_Hybrid', 'fueltype_Petrol']].values

dataset_train['choice_walk'] = (dataset_train['choice'] == 0).astype(int)
dataset_train['choice_cycle'] = (dataset_train['choice'] == 1).astype(int)
dataset_train['choice_pt'] = (dataset_train['choice'] == 2).astype(int)
dataset_train['choice_drive'] = (dataset_train['choice'] == 3).astype(int)
dataset_train['zeros'] = np.zeros_like(dataset_train['choice'])

train_data = np.stack(
    [dataset_train[['distance',  'zeros',    'zeros',    'zeros',    'dur_walking',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_walk']].values,
    dataset_train[['zeros',      'distance', 'zeros',    'zeros',    'zeros',        'dur_cycling',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_cycle']].values,
    dataset_train[['zeros',      'zeros',    'distance', 'zeros',    'zeros',        'zeros',        'dur_pt_access','zeros',        'dur_pt_rail',  'dur_pt_bus',   'dur_pt_int_waiting',   'dur_pt_int_walking',   'pt_n_interchanges','cost_transit', 'zeros',            'zeros',            'zeros',                    'choice_pt']].values,
    dataset_train[['zeros',      'zeros',    'zeros',    'distance', 'zeros',        'zeros',        'zeros',        'dur_driving',  'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'cost_driving_fuel','congestion_charge','driving_traffic_percent',  'choice_drive']].values]
    )

In [47]:
np.isnan(extra_data).sum()

0

In [48]:
np.save('models/keras_input_lpmc_train.npy', np.array(train_data.swapaxes(0, 1).swapaxes(1, 2), dtype=np.float32))
np.save('models/keras_input_lpmc_train_extra.npy', np.array(extra_data, dtype=np.float32))

In [56]:

extra_data = dataset_test[['age', 'female', 'start_time_linear', 'day_of_week', 'car_ownership', 'driving_license', 'purpose_B', 'purpose_HBE', 'purpose_HBO', 'purpose_HBW', 'purpose_NHBO', 'fueltype_Average', 'fueltype_Diesel', 'fueltype_Hybrid', 'fueltype_Petrol']].values

dataset_test['choice_walk'] = (dataset_test['choice'] == 0).astype(int)
dataset_test['choice_cycle'] = (dataset_test['choice'] == 1).astype(int)
dataset_test['choice_pt'] = (dataset_test['choice'] == 2).astype(int)
dataset_test['choice_drive'] = (dataset_test['choice'] == 3).astype(int)

dataset_test['zeros'] = np.zeros_like(dataset_test['choice'])

test_data = np.stack(
    [dataset_test[['distance',  'zeros',    'zeros',    'zeros',    'dur_walking',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_walk']].values,
    dataset_test[['zeros',      'distance', 'zeros',    'zeros',    'zeros',        'dur_cycling',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_cycle']].values,
    dataset_test[['zeros',      'zeros',    'distance', 'zeros',    'zeros',        'zeros',        'dur_pt_access','zeros',        'dur_pt_rail',  'dur_pt_bus',   'dur_pt_int_waiting',   'dur_pt_int_walking',   'pt_n_interchanges','cost_transit', 'zeros',            'zeros',            'zeros',                    'choice_pt']].values,
    dataset_test[['zeros',      'zeros',    'zeros',    'distance', 'zeros',        'zeros',        'zeros',        'dur_driving',  'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'cost_driving_fuel','congestion_charge','driving_traffic_percent',  'choice_drive']].values]
    )

In [57]:
np.save('models/keras_input_lpmc_test.npy', np.array(test_data.swapaxes(0, 1).swapaxes(1, 2), dtype=np.float32))
np.save('models/keras_input_lpmc_test_extra.npy', np.array(extra_data, dtype=np.float32))

In [62]:
test_data.shape

(4, 26320, 18)

In [63]:

# get all features
target = "choice"
features = [f for f in data_train.columns if f != target]

# get household ids
hh_id = np.array(data_train["household_id"].values)

# k folds sampled by households for cross validation
train_idx = []
test_idx = []

for train_i, test_i in stratified_group_k_fold(
    data_train[features], data_train["travel_mode"], hh_id, k=5
):
    train_idx.append(train_i)
    test_idx.append(test_i)

train_indices = zip(train_idx, test_idx)

for i, (train_idx, test_idx) in enumerate(train_indices):
    train_data = np.stack(
        [dataset_train[['distance',  'zeros',    'zeros',    'zeros',    'dur_walking',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_walk']].iloc[train_idx].values,
        dataset_train[['zeros',      'distance', 'zeros',    'zeros',    'zeros',        'dur_cycling',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_cycle']].iloc[train_idx].values,
        dataset_train[['zeros',      'zeros',    'distance', 'zeros',    'zeros',        'zeros',        'dur_pt_access','zeros',        'dur_pt_rail',  'dur_pt_bus',   'dur_pt_int_waiting',   'dur_pt_int_walking',   'pt_n_interchanges','cost_transit', 'zeros',            'zeros',            'zeros',                    'choice_pt']].iloc[train_idx].values,
        dataset_train[['zeros',      'zeros',    'zeros',    'distance', 'zeros',        'zeros',        'zeros',        'dur_driving',  'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'cost_driving_fuel','congestion_charge','driving_traffic_percent',  'choice_drive']].iloc[train_idx].values])

	
    extra_data = dataset_train[['age', 'female', 'start_time_linear', 'day_of_week', 'car_ownership', 'driving_license', 'purpose_B', 'purpose_HBE', 'purpose_HBO', 'purpose_HBW', 'purpose_NHBO', 'fueltype_Average', 'fueltype_Diesel', 'fueltype_Hybrid', 'fueltype_Petrol']].iloc[train_idx].values


    np.save(f'models/keras_input_lpmc_CVtrain_{i}.npy', np.array(train_data.swapaxes(0, 1).swapaxes(1, 2), dtype=np.float32))
    np.save(f'models/keras_input_lpmc_CVtrain_{i}_extra.npy', np.array(extra_data, dtype=np.float32))

    test_data = np.stack(
        [dataset_train[['distance',  'zeros',    'zeros',    'zeros',    'dur_walking',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_walk']].iloc[test_idx].values,
        dataset_train[['zeros',      'distance', 'zeros',    'zeros',    'zeros',        'dur_cycling',  'zeros',        'zeros',        'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'zeros',            'zeros',            'zeros',                    'choice_cycle']].iloc[test_idx].values,
        dataset_train[['zeros',      'zeros',    'distance', 'zeros',    'zeros',        'zeros',        'dur_pt_access','zeros',        'dur_pt_rail',  'dur_pt_bus',   'dur_pt_int_waiting',   'dur_pt_int_walking',   'pt_n_interchanges','cost_transit', 'zeros',            'zeros',            'zeros',                    'choice_pt']].iloc[test_idx].values,
        dataset_train[['zeros',      'zeros',    'zeros',    'distance', 'zeros',        'zeros',        'zeros',        'dur_driving',  'zeros',        'zeros',        'zeros',                'zeros',                'zeros',            'zeros',        'cost_driving_fuel','congestion_charge','driving_traffic_percent',  'choice_drive']].iloc[test_idx].values]
        )

    extra_data = dataset_train[['age', 'female', 'start_time_linear', 'day_of_week', 'car_ownership', 'driving_license', 'purpose_B', 'purpose_HBE', 'purpose_HBO', 'purpose_HBW', 'purpose_NHBO', 'fueltype_Average', 'fueltype_Diesel', 'fueltype_Hybrid', 'fueltype_Petrol']].iloc[test_idx].values

    np.save(f'models/keras_input_lpmc_CVtest_{i}.npy', np.array(test_data.swapaxes(0, 1).swapaxes(1, 2), dtype=np.float32))
    np.save(f'models/keras_input_lpmc_CVtest_{i}_extra.npy', np.array(extra_data, dtype=np.float32))


[[[0.778    0.       0.       ... 0.       0.       0.      ]
  [0.778    0.       0.       ... 0.       0.       0.      ]
  [4.562    0.       0.       ... 0.       0.       0.      ]
  ...
  [5.004    0.       0.       ... 0.       0.       0.      ]
  [7.237    0.       0.       ... 0.       0.       0.      ]
  [7.237    0.       0.       ... 0.       0.       0.      ]]

 [[0.       0.778    0.       ... 0.       0.       0.      ]
  [0.       0.778    0.       ... 0.       0.       0.      ]
  [0.       4.562    0.       ... 0.       0.       0.      ]
  ...
  [0.       5.004    0.       ... 0.       0.       0.      ]
  [0.       7.237    0.       ... 0.       0.       0.      ]
  [0.       7.237    0.       ... 0.       0.       0.      ]]

 [[0.       0.       0.778    ... 0.       0.       0.      ]
  [0.       0.       0.778    ... 0.       0.       0.      ]
  [0.       0.       4.562    ... 0.       0.       0.      ]
  ...
  [0.       0.       5.004    ... 0.       0.   